In [1]:
import pandas as pd
import numpy as np
import requests
import json
import geopy.distance
from geopy.distance import geodesic
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics, ensemble
from sklearn.metrics import r2_score


In [2]:
train = pd.read_csv('../input/amazon-august/train.csv')
test = pd.read_csv('../input/amazon-august/test.csv')

In [3]:
train = train.iloc[:,1:]
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30,11:45,Sunny,High,2.0,Snack,motorcycle,0.0,No,Urban,24.0
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45,19:50,Stormy,Jam,2.0,Snack,scooter,1.0,No,Metropolitian,33.0
2,0x20f7,BANGRES18DEL01,37.0,4.4,12.913041,77.683237,12.953041,77.723237,19-03-2022,13:50,13:55,Windy,High,0.0,Buffet,motorcycle,2.0,No,Metropolitian,42.0
3,0x7804,HYDRES13DEL02,28.000000,4.900000,17.431477,78.400350,17.451477,78.420350,07-03-2022,10:60,11:15,Sandstorms,Low,1.0,Meal,scooter,1.000000,No,Urban,19.0
4,0x7faf,RANCHIRES07DEL01,37.0,4.6,23.359407,85.325055,23.429407,85.395055,18-03-2022,23:50,23:60,Stormy,Low,0.0,Meal,motorcycle,1.0,No,Metropolitian,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0xd82f,AGRRES11DEL02,34.0,4.900000,27.163303,78.057044,27.213303,78.107044,11-02-2022,18:20,18:25,Cloudy,Medium,2.0,Snack,scooter,0.0,No,Metropolitian,19.0
45589,0x98f6,PUNERES18DEL03,20.0,4.8,18.593481,73.785901,18.653481,73.845901,05-04-2022,17:60,18:05,Sunny,Medium,1.0,Drinks,motorcycle,1.000000,No,Metropolitian,14.0
45590,0x744,COIMBRES06DEL01,NaN,NaN,11.021278,76.995017,11.061278,77.035017,11-03-2022,NaN,13:10,NaN,NaN,3.0,Snack,electric_scooter,1.000000,No,Metropolitian,37.0
45591,0x419b,JAPRES03DEL03,23.0,4.900000,26.913483,75.803139,27.053483,75.943139,25-03-2022,19:50,19:60,Cloudy,Jam,2.0,Meal,motorcycle,1.0,No,Metropolitian,42.0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          44651 non-null  object 
 3   Delivery_person_Ratings      44611 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  44738 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weather conditions           45217 non-null  object 
 12  Road_traffic_density         45222 non-null  object 
 13  Vehicle_conditio

In [5]:
train.describe()

,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,Time_taken (min)
count,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000
mean,17.017729,70.231332,17.465186,70.845702,1.023359,26.294607
std,8.185109,22.883647,7.335122,21.118812,0.839065,9.383806
min,-30.905562,-88.366217,0.010000,0.010000,0.000000,10.000000
25%,12.933284,73.170000,12.988453,73.280000,0.000000,19.000000
50%,18.546947,75.898497,18.633934,76.002574,1.000000,26.000000
75%,22.728163,78.044095,22.785049,78.107044,2.000000,32.000000
max,30.914057,88.433452,31.054057,88.563452,3.000000,54.000000


In [6]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30,11:45,Sunny,High,2.0,Snack,motorcycle,0.0,No,Urban,24.0
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45,19:50,Stormy,Jam,2.0,Snack,scooter,1.0,No,Metropolitian,33.0
2,0x20f7,BANGRES18DEL01,37.0,4.4,12.913041,77.683237,12.953041,77.723237,19-03-2022,13:50,13:55,Windy,High,0.0,Buffet,motorcycle,2.0,No,Metropolitian,42.0
3,0x7804,HYDRES13DEL02,28.000000,4.900000,17.431477,78.400350,17.451477,78.420350,07-03-2022,10:60,11:15,Sandstorms,Low,1.0,Meal,scooter,1.000000,No,Urban,19.0
4,0x7faf,RANCHIRES07DEL01,37.0,4.6,23.359407,85.325055,23.429407,85.395055,18-03-2022,23:50,23:60,Stormy,Low,0.0,Meal,motorcycle,1.0,No,Metropolitian,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0xd82f,AGRRES11DEL02,34.0,4.900000,27.163303,78.057044,27.213303,78.107044,11-02-2022,18:20,18:25,Cloudy,Medium,2.0,Snack,scooter,0.0,No,Metropolitian,19.0
45589,0x98f6,PUNERES18DEL03,20.0,4.8,18.593481,73.785901,18.653481,73.845901,05-04-2022,17:60,18:05,Sunny,Medium,1.0,Drinks,motorcycle,1.000000,No,Metropolitian,14.0
45590,0x744,COIMBRES06DEL01,NaN,NaN,11.021278,76.995017,11.061278,77.035017,11-03-2022,NaN,13:10,NaN,NaN,3.0,Snack,electric_scooter,1.000000,No,Metropolitian,37.0
45591,0x419b,JAPRES03DEL03,23.0,4.900000,26.913483,75.803139,27.053483,75.943139,25-03-2022,19:50,19:60,Cloudy,Jam,2.0,Meal,motorcycle,1.0,No,Metropolitian,42.0


In [7]:
print(train.apply(lambda col: col.unique()))

ID                             [ 0x4607,  0xb379,  0x20f7, 0x7804,  0x7faf,  ...
Delivery_person_ID             [ INDORES13DEL02,  BANGRES18DEL02,  BANGRES18D...
Delivery_person_Age            [37.0, 34.0, 28.000000, 20.0, 32.0, 34.000000,...
Delivery_person_Ratings        [4.9, 4.5, 4.4,  4.900000, 4.6, 4.7,  4.800000...
Restaurant_latitude            [22.745049, 12.913041, 17.431477, 23.359407, 1...
Restaurant_longitude           [75.892471, 77.683237, 78.40035, 85.325055, 77...
Delivery_location_latitude     [22.765049, 13.043041, 12.953041, 17.451477, 2...
Delivery_location_longitude    [75.912471, 77.813237, 77.723237, 78.42035, 85...
Order_Date                     [ 19-03-2022,  25-03-2022, 07-03-2022,  18-03-...
Time_Orderd                    [ 11:30,  19:45,  13:50, 10:60,  23:50,  17:60...
Time_Order_picked              [ 11:45,  19:50,  13:55, 11:15,  23:60,  18:10...
Weather conditions             [Sunny,  Stormy, Windy, Sandstorms, Stormy,  F...
Road_traffic_density        

In [8]:
train = train.replace(r"^ +|", r"", regex=True)
test = test.replace(r"^ +|", r"", regex=True)
#train[['Time_Order_picked','Weather conditions','Road_traffic_density','Type_of_vehicle','Festival','City']].replace(" ", "", limit = 1, inplace = True)

In [9]:
# call the OSMR API

lon_1 = train['Restaurant_longitude'][3]
lat_1 = train['Restaurant_latitude'][3]
lon_2 = train['Delivery_location_longitude'][3]
lat_2 = train['Delivery_location_latitude'][3]

r = requests.get(f"http://router.project-osrm.org/route/v1/bike/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")
routes = json.loads(r.content)
route_1 = routes.get("routes")[0].get("distance")/1000

print(lon_1,lat_1, lon_2, lat_2)

78.40035 17.431477 78.42035 17.451477


In [10]:
route_1

4.6258

In [11]:
def get_distance(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    coords_1 = (pickup_lon, pickup_lat)
    coords_2 = (dropoff_lon, dropoff_lat)
    return geodesic(coords_1, coords_2).km

In [12]:
train['Distance'] = train.apply(lambda x: get_distance(x['Restaurant_longitude'], 
                                            x['Restaurant_latitude'],
                                            x['Delivery_location_longitude'], 
                                            x['Delivery_location_latitude']), axis=1)

test['Distance'] = test.apply(lambda x: get_distance(x['Restaurant_longitude'], 
                                            x['Restaurant_latitude'],
                                            x['Delivery_location_longitude'], 
                                            x['Delivery_location_latitude']), axis=1)

In [13]:
train.describe()

,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,Time_taken (min),Distance
count,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000
mean,17.017729,70.231332,17.465186,70.845702,1.023359,26.294607,74.722763
std,8.185109,22.883647,7.335122,21.118812,0.839065,9.383806,1037.566825
min,-30.905562,-88.366217,0.010000,0.010000,0.000000,10.000000,1.117346
25%,12.933284,73.170000,12.988453,73.280000,0.000000,19.000000,3.490530
50%,18.546947,75.898497,18.633934,76.002574,1.000000,26.000000,6.950218
75%,22.728163,78.044095,22.785049,78.107044,2.000000,32.000000,10.342382
max,30.914057,88.433452,31.054057,88.563452,3.000000,54.000000,19680.354238


In [14]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    r = requests.get(f"http://router.project-osrm.org/route/v1/bike/{pickup_lon},{pickup_lat};{dropoff_lon},{dropoff_lat}?overview=false""")
    routes = json.loads(r.content)
    dis = routes.get("routes")[0].get("distance")/1000
    return dis

In [15]:
pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = 78.40035, 17.431477, 78.42035, 17.451477
test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
test_route

4.6258

lon1 = np.random.uniform(-117.4,-118, 50000).round(6)
lon2 = np.random.uniform(-117.4,-118, 50000).round(6)
lat1 = np.random.uniform(33.6,33.8, 50000).round(6)
lat2 = np.random.uniform(33.6,33.8, 50000).round(6)
train = pd.DataFrame({'pickup_lon': lon1,
              'pickup_lat': lat1,
              'dropoff_lon': lon2,
              'dropoff_lat': lat2,
             })

train['routes'] = train.apply(lambda x: get_route(x['pickup_lon'], 
                                            x['pickup_lat'],
                                            x['dropoff_lon'],
                                            x['dropoff_lat']), axis=1)

In [16]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),Distance
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30,...,Sunny,High,2.0,Snack,motorcycle,0.0,No,Urban,24.0,2.297864
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45,...,Stormy,Jam,2.0,Snack,scooter,1.0,No,Metropolitian,33.0,14.836986
2,0x20f7,BANGRES18DEL01,37.0,4.4,12.913041,77.683237,12.953041,77.723237,19-03-2022,13:50,...,Windy,High,0.0,Buffet,motorcycle,2.0,No,Metropolitian,42.0,4.565927
3,0x7804,HYDRES13DEL02,28.000000,4.900000,17.431477,78.400350,17.451477,78.420350,07-03-2022,10:60,...,Sandstorms,Low,1.0,Meal,scooter,1.000000,No,Urban,19.0,2.277609
4,0x7faf,RANCHIRES07DEL01,37.0,4.6,23.359407,85.325055,23.429407,85.395055,18-03-2022,23:50,...,Stormy,Low,0.0,Meal,motorcycle,1.0,No,Metropolitian,25.0,7.843600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0xd82f,AGRRES11DEL02,34.0,4.900000,27.163303,78.057044,27.213303,78.107044,11-02-2022,18:20,...,Cloudy,Medium,2.0,Snack,scooter,0.0,No,Metropolitian,19.0,5.700149
45589,0x98f6,PUNERES18DEL03,20.0,4.8,18.593481,73.785901,18.653481,73.845901,05-04-2022,17:60,...,Sunny,Medium,1.0,Drinks,motorcycle,1.000000,No,Metropolitian,14.0,6.951888
45590,0x744,COIMBRES06DEL01,NaN,NaN,11.021278,76.995017,11.061278,77.035017,11-03-2022,NaN,...,NaN,NaN,3.0,Snack,electric_scooter,1.000000,No,Metropolitian,37.0,4.576893
45591,0x419b,JAPRES03DEL03,23.0,4.900000,26.913483,75.803139,27.053483,75.943139,25-03-2022,19:50,...,Cloudy,Jam,2.0,Meal,motorcycle,1.0,No,Metropolitian,42.0,16.086837


train['route'] = train.apply(lambda x: get_route(x['Restaurant_longitude'], 
                                            x['Restaurant_latitude'],
                                            x['Delivery_location_longitude'], 
                                            x['Delivery_location_latitude']), axis=1)

In [17]:
train.drop(['Restaurant_latitude', 'Delivery_location_latitude','Restaurant_longitude','Delivery_location_longitude','Time_Orderd','Type_of_order'], axis=1, inplace=True)
test.drop(['Restaurant_latitude', 'Delivery_location_latitude','Restaurant_longitude','Delivery_location_longitude','Time_Orderd','Type_of_order'], axis=1, inplace=True)

In [18]:

train.insert(4, "Distance", train.pop("Distance"))
test.insert(4, "Distance", test.pop("Distance"))

In [19]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Distance,Order_Date,Time_Order_picked,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0x4607,INDORES13DEL02,37.0,4.9,2.297864,19-03-2022,11:45,Sunny,High,2.0,motorcycle,0.0,No,Urban,24.0
1,0xb379,BANGRES18DEL02,34.0,4.5,14.836986,25-03-2022,19:50,Stormy,Jam,2.0,scooter,1.0,No,Metropolitian,33.0
2,0x20f7,BANGRES18DEL01,37.0,4.4,4.565927,19-03-2022,13:55,Windy,High,0.0,motorcycle,2.0,No,Metropolitian,42.0
3,0x7804,HYDRES13DEL02,28.000000,4.900000,2.277609,07-03-2022,11:15,Sandstorms,Low,1.0,scooter,1.000000,No,Urban,19.0
4,0x7faf,RANCHIRES07DEL01,37.0,4.6,7.843600,18-03-2022,23:60,Stormy,Low,0.0,motorcycle,1.0,No,Metropolitian,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0xd82f,AGRRES11DEL02,34.0,4.900000,5.700149,11-02-2022,18:25,Cloudy,Medium,2.0,scooter,0.0,No,Metropolitian,19.0
45589,0x98f6,PUNERES18DEL03,20.0,4.8,6.951888,05-04-2022,18:05,Sunny,Medium,1.0,motorcycle,1.000000,No,Metropolitian,14.0
45590,0x744,COIMBRES06DEL01,NaN,NaN,4.576893,11-03-2022,13:10,NaN,NaN,3.0,electric_scooter,1.000000,No,Metropolitian,37.0
45591,0x419b,JAPRES03DEL03,23.0,4.900000,16.086837,25-03-2022,19:60,Cloudy,Jam,2.0,motorcycle,1.0,No,Metropolitian,42.0


In [20]:
test

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Distance,Order_Date,Time_Order_picked,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City
0,0x2318,COIMBRES13DEL01,NaN,NaN,4.577196,30-03-2022,15:05,NaN,NaN,3.0,electric_scooter,1.000000,No,Metropolitian
1,0x3474,BANGRES15DEL01,28.0,4.6,12.554229,29-03-2022,20:35,Windy,Jam,0.0,motorcycle,1.0,No,Metropolitian
2,0x9420,JAPRES09DEL03,23.0,4.500000,10.343089,10-03-2022,19:45,Stormy,Jam,0.0,motorcycle,1.0,No,Metropolitian
3,0x72ee,JAPRES07DEL03,21.0,4.800000,10.341179,02-04-2022,17:20,Fog,Medium,1.0,scooter,1.0,No,Metropolitian
4,0xa759,CHENRES19DEL01,31.0,4.6,12.456925,27-03-2022,18:40,Sunny,Medium,2.0,scooter,1.000000,No,Metropolitian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11394,0x6909,JAPRES01DEL01,35.0,4.600000,12.640017,27-03-2022,21:45,Sunny,Jam,1.0,scooter,1.0,No,Metropolitian
11395,0x443b,JAPRES11DEL01,33.000000,4.900000,1.149391,11-03-2022,11:45,Sandstorms,High,1.0,scooter,1.000000,No,Metropolitian
11396,0x1ea5,SURRES11DEL03,NaN,NaN,6.982700,11-03-2022,21:05,NaN,NaN,3.0,scooter,1.0,No,Metropolitian
11397,0x22d4,VADRES03DEL02,27.0,4.70,15.100142,06-03-2022,18:40,Sandstorms,Medium,0.0,motorcycle,0.0,No,Metropolitian


In [21]:
train.describe()

,Distance,Vehicle_condition,Time_taken (min)
count,45593.000000,45593.000000,45593.000000
mean,74.722763,1.023359,26.294607
std,1037.566825,0.839065,9.383806
min,1.117346,0.000000,10.000000
25%,3.490530,0.000000,19.000000
50%,6.950218,1.000000,26.000000
75%,10.342382,2.000000,32.000000
max,19680.354238,3.000000,54.000000


In [22]:
train.describe(include = 'O')

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Order_Date,Time_Order_picked,Weather conditions,Road_traffic_density,Type_of_vehicle,multiple_deliveries,Festival,City
count,45593,45593,44651,44611,45593,45593,45217,45222,45593,45062,45481,45052
unique,45593,1320,85,80,44,193,7,5,4,17,3,4
top,0x4607,JAPRES11DEL02,35.0,4.8,15-03-2022,21:30,Fog,Low,motorcycle,1.0,No,Metropolitian
freq,1,67,1790,3532,1192,496,7654,15477,26435,20227,44469,34093


In [23]:
train.replace("NaN", np.NaN, inplace = True)
test.replace("NaN", np.NaN, inplace = True)

In [24]:
train.isnull().sum()

ID                            0
Delivery_person_ID            0
Delivery_person_Age        1854
Delivery_person_Ratings    1908
Distance                      0
Order_Date                    0
Time_Order_picked             0
Weather conditions          616
Road_traffic_density        601
Vehicle_condition             0
Type_of_vehicle               0
multiple_deliveries         993
Festival                    228
City                       1200
Time_taken (min)              0
dtype: int64

In [25]:
test.isnull().sum()

ID                           0
Delivery_person_ID           0
Delivery_person_Age        491
Delivery_person_Ratings    507
Distance                     0
Order_Date                   0
Time_Order_picked            0
Weather conditions         158
Road_traffic_density       154
Vehicle_condition            0
Type_of_vehicle              0
multiple_deliveries        238
Festival                    65
City                       324
dtype: int64

In [26]:
train[['Delivery_person_Age', 'Delivery_person_Ratings','multiple_deliveries']] = train[['Delivery_person_Age', 'Delivery_person_Ratings','multiple_deliveries']].apply(pd.to_numeric)
test[['Delivery_person_Age', 'Delivery_person_Ratings','multiple_deliveries']] = test[['Delivery_person_Age', 'Delivery_person_Ratings','multiple_deliveries']].apply(pd.to_numeric)

In [27]:
train.describe(include = 'O')

,ID,Delivery_person_ID,Order_Date,Time_Order_picked,Weather conditions,Road_traffic_density,Type_of_vehicle,Festival,City
count,45593,45593,45593,45593,44977,44992,45593,45365,44393
unique,45593,1320,44,193,6,4,4,2,3
top,0x4607,JAPRES11DEL02,15-03-2022,21:30,Fog,Low,motorcycle,No,Metropolitian
freq,1,67,1192,496,7654,15477,26435,44469,34093


In [28]:
train.describe()

,Delivery_person_Age,Delivery_person_Ratings,Distance,Vehicle_condition,multiple_deliveries,Time_taken (min)
count,43739.000000,43685.000000,45593.000000,45593.000000,44600.000000,45593.000000
mean,29.567137,4.633780,74.722763,1.023359,0.744664,26.294607
std,5.815155,0.334716,1037.566825,0.839065,0.572473,9.383806
min,15.000000,1.000000,1.117346,0.000000,0.000000,10.000000
25%,25.000000,4.500000,3.490530,0.000000,0.000000,19.000000
50%,30.000000,4.700000,6.950218,1.000000,1.000000,26.000000
75%,35.000000,4.900000,10.342382,2.000000,1.000000,32.000000
max,50.000000,6.000000,19680.354238,3.000000,3.000000,54.000000


def days_months(train):
    d = pd.to_datetime(train['Order_Date'])
    
    #train['dayofweek'] = d.dt.dayofweek
    train['month_name'] = d.dt.month_name()
    train['day_name'] = d.dt.day_name()
    
    
days_months(X)

In [29]:
train.isnull().sum()

ID                            0
Delivery_person_ID            0
Delivery_person_Age        1854
Delivery_person_Ratings    1908
Distance                      0
Order_Date                    0
Time_Order_picked             0
Weather conditions          616
Road_traffic_density        601
Vehicle_condition             0
Type_of_vehicle               0
multiple_deliveries         993
Festival                    228
City                       1200
Time_taken (min)              0
dtype: int64

In [30]:
train['Delivery_person_Age'] = train['Delivery_person_Age'].fillna(30)
train['multiple_deliveries'] = train['multiple_deliveries'].fillna(0)
train['Delivery_person_Ratings'] = train['Delivery_person_Ratings'].fillna(train['Delivery_person_Ratings'].mean())
train[['Weather conditions','Road_traffic_density','Type_of_vehicle','Festival','City']] = train[['Weather conditions','Road_traffic_density','Type_of_vehicle','Festival','City']].fillna(train[['Weather conditions','Road_traffic_density','Type_of_vehicle','Festival','City']].mode().iloc[0])


test['Delivery_person_Age'] = test['Delivery_person_Age'].fillna(30)
test['multiple_deliveries'] = test['multiple_deliveries'].fillna(0)
test['Delivery_person_Ratings'] = test['Delivery_person_Ratings'].fillna(test['Delivery_person_Ratings'].mean())
test[['Weather conditions','Road_traffic_density','Type_of_vehicle','Festival','City']] = test[['Weather conditions','Road_traffic_density','Type_of_vehicle','Festival','City']].fillna(test[['Weather conditions','Road_traffic_density','Type_of_vehicle','Festival','City']].mode().iloc[0])

In [31]:
train.isnull().sum()

ID                         0
Delivery_person_ID         0
Delivery_person_Age        0
Delivery_person_Ratings    0
Distance                   0
Order_Date                 0
Time_Order_picked          0
Weather conditions         0
Road_traffic_density       0
Vehicle_condition          0
Type_of_vehicle            0
multiple_deliveries        0
Festival                   0
City                       0
Time_taken (min)           0
dtype: int64

In [32]:
train.describe(include = 'O')

,ID,Delivery_person_ID,Order_Date,Time_Order_picked,Weather conditions,Road_traffic_density,Type_of_vehicle,Festival,City
count,45593,45593,45593,45593,45593,45593,45593,45593,45593
unique,45593,1320,44,193,6,4,4,2,3
top,0x4607,JAPRES11DEL02,15-03-2022,21:30,Fog,Low,motorcycle,No,Metropolitian
freq,1,67,1192,496,8270,16078,26435,44697,35293


In [33]:
train.describe()

,Delivery_person_Age,Delivery_person_Ratings,Distance,Vehicle_condition,multiple_deliveries,Time_taken (min)
count,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000
mean,29.584739,4.633780,74.722763,1.023359,0.728445,26.294607
std,5.696333,0.327638,1037.566825,0.839065,0.576543,9.383806
min,15.000000,1.000000,1.117346,0.000000,0.000000,10.000000
25%,25.000000,4.600000,3.490530,0.000000,0.000000,19.000000
50%,30.000000,4.700000,6.950218,1.000000,1.000000,26.000000
75%,34.000000,4.800000,10.342382,2.000000,1.000000,32.000000
max,50.000000,6.000000,19680.354238,3.000000,3.000000,54.000000


In [34]:
print(train.apply(lambda col: col.unique()))

ID                         [0x4607, 0xb379, 0x20f7, 0x7804, 0x7faf, 0x521...
Delivery_person_ID         [INDORES13DEL02, BANGRES18DEL02, BANGRES18DEL0...
Delivery_person_Age        [37.0, 34.0, 28.0, 20.0, 32.0, 24.0, 23.0, 22....
Delivery_person_Ratings    [4.9, 4.5, 4.4, 4.6, 4.7, 4.8, 5.0, 4.2, 4.3, ...
Distance                   [2.2978637034294684, 14.836985580032627, 4.565...
Order_Date                 [19-03-2022, 25-03-2022, 07-03-2022, 18-03-202...
Time_Order_picked          [11:45, 19:50, 13:55, 11:15, 23:60, 18:10, 22:...
Weather conditions           [Sunny, Stormy, Windy, Sandstorms, Fog, Cloudy]
Road_traffic_density                                [High, Jam, Low, Medium]
Vehicle_condition                                       [2.0, 0.0, 1.0, 3.0]
Type_of_vehicle             [motorcycle, scooter, electric_scooter, bicycle]
multiple_deliveries                                     [0.0, 1.0, 2.0, 3.0]
Festival                                                           [No, Yes]

In [35]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Distance,Order_Date,Time_Order_picked,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0x4607,INDORES13DEL02,37.0,4.90000,2.297864,19-03-2022,11:45,Sunny,High,2.0,motorcycle,0.0,No,Urban,24.0
1,0xb379,BANGRES18DEL02,34.0,4.50000,14.836986,25-03-2022,19:50,Stormy,Jam,2.0,scooter,1.0,No,Metropolitian,33.0
2,0x20f7,BANGRES18DEL01,37.0,4.40000,4.565927,19-03-2022,13:55,Windy,High,0.0,motorcycle,2.0,No,Metropolitian,42.0
3,0x7804,HYDRES13DEL02,28.0,4.90000,2.277609,07-03-2022,11:15,Sandstorms,Low,1.0,scooter,1.0,No,Urban,19.0
4,0x7faf,RANCHIRES07DEL01,37.0,4.60000,7.843600,18-03-2022,23:60,Stormy,Low,0.0,motorcycle,1.0,No,Metropolitian,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0xd82f,AGRRES11DEL02,34.0,4.90000,5.700149,11-02-2022,18:25,Cloudy,Medium,2.0,scooter,0.0,No,Metropolitian,19.0
45589,0x98f6,PUNERES18DEL03,20.0,4.80000,6.951888,05-04-2022,18:05,Sunny,Medium,1.0,motorcycle,1.0,No,Metropolitian,14.0
45590,0x744,COIMBRES06DEL01,30.0,4.63378,4.576893,11-03-2022,13:10,Fog,Low,3.0,electric_scooter,1.0,No,Metropolitian,37.0
45591,0x419b,JAPRES03DEL03,23.0,4.90000,16.086837,25-03-2022,19:60,Cloudy,Jam,2.0,motorcycle,1.0,No,Metropolitian,42.0


In [36]:
train.sort_values('Distance', ascending=False)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Distance,Order_Date,Time_Order_picked,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
40177,0xbf01,KOLRES06DEL02,50.0,6.00000,19680.354238,18-02-2022,21:50,Fog,Low,3.0,scooter,1.0,No,Metropolitian,28.0
16106,0xc014,KOLRES15DEL01,30.0,4.63378,19675.578410,16-02-2022,20:45,Stormy,Jam,0.0,motorcycle,3.0,No,Metropolitian,46.0
12232,0xbf10,KOLRES010DEL02,30.0,4.63378,19671.319584,18-02-2022,19:30,Stormy,Jam,1.0,motorcycle,1.0,No,Metropolitian,22.0
26941,0xc012,KOLRES08DEL01,30.0,4.63378,19664.515267,11-02-2022,8:30,Cloudy,Low,2.0,scooter,1.0,No,Urban,15.0
1858,0x3ef,RANCHIRES13DEL01,50.0,6.00000,19055.095776,20-03-2022,17:55,Fog,Low,3.0,bicycle,1.0,No,Metropolitian,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32680,0xc678,KOLRES18DEL01,33.0,4.60000,1.117346,11-02-2022,11:55,Stormy,High,1.0,scooter,1.0,No,Metropolitian,26.0
22560,0xc57b,KOLRES17DEL01,35.0,4.90000,1.117346,13-02-2022,10:45,Stormy,Low,1.0,motorcycle,1.0,No,Metropolitian,22.0
16569,0xd4d6,KOLRES17DEL01,32.0,4.70000,1.117346,17-02-2022,10:30,Sandstorms,Low,1.0,motorcycle,1.0,No,Urban,26.0
13648,0xc1e5,KOLRES17DEL01,23.0,5.00000,1.117346,11-02-2022,9:35,Sandstorms,Low,2.0,electric_scooter,1.0,No,Metropolitian,16.0


def weather(train):
    def weather_helper(s):
        if s in ('Sunny','Fog','Cloudy'):
            return 1
        return 0
    
    train['Favourable_weather'] = train['Weather conditions'].apply(weather_helper)
    train['Unfavourable_weather'] = train['Weather conditions'].apply(lambda x: 1 - weather_helper(x))
weather(train)

def vehicle(train):
    def vehicle_helper(s):
        if s in ('bicycle'):
            return 0
        return 1
    
    train['Motor_vehicle'] = train['Type_of_vehicle'].apply(vehicle_helper)
    train['Non_motor_vehicle'] = train['Type_of_vehicle'].apply(lambda x: 1 - vehicle_helper(x))  
vehicle(train)

In [37]:
try_train = train
try_test = test

In [38]:
#First_try


def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)


features_to_encode = ['Weather conditions','Road_traffic_density', 'Type_of_vehicle',
                      'City']
for feature in features_to_encode:
    try_train = encode_and_bind(try_train, feature)
    
for feature in features_to_encode:
    try_test = encode_and_bind(try_test, feature)

In [39]:
train.corr()

,Delivery_person_Age,Delivery_person_Ratings,Distance,Vehicle_condition,multiple_deliveries,Time_taken (min)
Delivery_person_Age,1.000000,-0.067881,0.023910,0.004788,0.113326,0.292861
Delivery_person_Ratings,-0.067881,1.000000,0.001495,0.033119,-0.114943,-0.331115
Distance,0.023910,0.001495,1.000000,0.092853,0.000748,-0.002137
Vehicle_condition,0.004788,0.033119,0.092853,1.000000,-0.095840,-0.234398
multiple_deliveries,0.113326,-0.114943,0.000748,-0.095840,1.000000,0.386497
Time_taken (min),0.292861,-0.331115,-0.002137,-0.234398,0.386497,1.000000


In [40]:
train['Road_traffic_density'].value_counts()

Low       16078
Jam       14143
Medium    10947
High       4425
Name: Road_traffic_density, dtype: int64

In [41]:
train['Weather conditions'] = train['Weather conditions'].apply(lambda x: 0 if x in ('Fog','Cloudy')  else 1)
train['Type_of_vehicle'] = train['Type_of_vehicle'].apply(lambda x: 0 if x in ('bicycle')  else 1)
train['Festival'] = train['Festival'].apply(lambda x: 0 if x in ('No')  else 1)
train['City'] = train['City'].apply(lambda x: 0 if x in ('Metropolitian')  else 1)


test['Weather conditions'] = test['Weather conditions'].apply(lambda x: 0 if x in ('Fog','Cloudy')  else 1)
test['Type_of_vehicle'] = test['Type_of_vehicle'].apply(lambda x: 0 if x in ('bicycle')  else 1)
test['Festival'] = test['Festival'].apply(lambda x: 0 if x in ('No')  else 1)
test['City'] = test['City'].apply(lambda x: 0 if x in ('Metropolitian')  else 1)

In [42]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Distance,Order_Date,Time_Order_picked,Weather conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0x4607,INDORES13DEL02,37.0,4.90000,2.297864,19-03-2022,11:45,1,High,2.0,1,0.0,0,1,24.0
1,0xb379,BANGRES18DEL02,34.0,4.50000,14.836986,25-03-2022,19:50,1,Jam,2.0,1,1.0,0,0,33.0
2,0x20f7,BANGRES18DEL01,37.0,4.40000,4.565927,19-03-2022,13:55,1,High,0.0,1,2.0,0,0,42.0
3,0x7804,HYDRES13DEL02,28.0,4.90000,2.277609,07-03-2022,11:15,1,Low,1.0,1,1.0,0,1,19.0
4,0x7faf,RANCHIRES07DEL01,37.0,4.60000,7.843600,18-03-2022,23:60,1,Low,0.0,1,1.0,0,0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0xd82f,AGRRES11DEL02,34.0,4.90000,5.700149,11-02-2022,18:25,0,Medium,2.0,1,0.0,0,0,19.0
45589,0x98f6,PUNERES18DEL03,20.0,4.80000,6.951888,05-04-2022,18:05,1,Medium,1.0,1,1.0,0,0,14.0
45590,0x744,COIMBRES06DEL01,30.0,4.63378,4.576893,11-03-2022,13:10,0,Low,3.0,1,1.0,0,0,37.0
45591,0x419b,JAPRES03DEL03,23.0,4.90000,16.086837,25-03-2022,19:60,0,Jam,2.0,1,1.0,0,0,42.0


In [43]:
train.corr()

,Delivery_person_Age,Delivery_person_Ratings,Distance,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
Delivery_person_Age,1.000000,-0.067881,0.023910,0.000213,0.004788,-0.004813,0.113326,0.057092,-0.057573,0.292861
Delivery_person_Ratings,-0.067881,1.000000,0.001495,-0.031262,0.033119,0.033839,-0.114943,-0.058092,0.050962,-0.331115
Distance,0.023910,0.001495,1.000000,-0.059690,0.092853,-0.151594,0.000748,0.010466,0.002337,-0.002137
Weather conditions,0.000213,-0.031262,-0.059690,1.000000,-0.039151,0.053056,-0.074197,-0.070516,0.028958,-0.196378
Vehicle_condition,0.004788,0.033119,0.092853,-0.039151,1.000000,-0.091047,-0.095840,-0.079085,0.033030,-0.234398
Type_of_vehicle,-0.004813,0.033839,-0.151594,0.053056,-0.091047,1.000000,-0.001445,-0.002717,0.003210,-0.000543
multiple_deliveries,0.113326,-0.114943,0.000748,-0.074197,-0.095840,-0.001445,1.000000,0.141512,-0.059768,0.386497
Festival,0.057092,-0.058092,0.010466,-0.070516,-0.079085,-0.002717,0.141512,1.000000,-0.019051,0.290046
City,-0.057573,0.050962,0.002337,0.028958,0.033030,0.003210,-0.059768,-0.019051,1.000000,-0.166074
Time_taken (min),0.292861,-0.331115,-0.002137,-0.196378,-0.234398,-0.000543,0.386497,0.290046,-0.166074,1.000000


In [44]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res.drop([feature_to_encode], axis=1, inplace = True)
    return(res)

train = encode_and_bind(train, 'Road_traffic_density')
test = encode_and_bind(test, 'Road_traffic_density')

In [45]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Distance,Order_Date,Time_Order_picked,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),Road_traffic_density_High,Road_traffic_density_Jam,Road_traffic_density_Low,Road_traffic_density_Medium
0,0x4607,INDORES13DEL02,37.0,4.90000,2.297864,19-03-2022,11:45,1,2.0,1,0.0,0,1,24.0,1,0,0,0
1,0xb379,BANGRES18DEL02,34.0,4.50000,14.836986,25-03-2022,19:50,1,2.0,1,1.0,0,0,33.0,0,1,0,0
2,0x20f7,BANGRES18DEL01,37.0,4.40000,4.565927,19-03-2022,13:55,1,0.0,1,2.0,0,0,42.0,1,0,0,0
3,0x7804,HYDRES13DEL02,28.0,4.90000,2.277609,07-03-2022,11:15,1,1.0,1,1.0,0,1,19.0,0,0,1,0
4,0x7faf,RANCHIRES07DEL01,37.0,4.60000,7.843600,18-03-2022,23:60,1,0.0,1,1.0,0,0,25.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0xd82f,AGRRES11DEL02,34.0,4.90000,5.700149,11-02-2022,18:25,0,2.0,1,0.0,0,0,19.0,0,0,0,1
45589,0x98f6,PUNERES18DEL03,20.0,4.80000,6.951888,05-04-2022,18:05,1,1.0,1,1.0,0,0,14.0,0,0,0,1
45590,0x744,COIMBRES06DEL01,30.0,4.63378,4.576893,11-03-2022,13:10,0,3.0,1,1.0,0,0,37.0,0,0,1,0
45591,0x419b,JAPRES03DEL03,23.0,4.90000,16.086837,25-03-2022,19:60,0,2.0,1,1.0,0,0,42.0,0,1,0,0


In [46]:
train.corr()

,Delivery_person_Age,Delivery_person_Ratings,Distance,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),Road_traffic_density_High,Road_traffic_density_Jam,Road_traffic_density_Low,Road_traffic_density_Medium
Delivery_person_Age,1.000000,-0.067881,0.023910,0.000213,0.004788,-0.004813,0.113326,0.057092,-0.057573,0.292861,0.003415,-0.001831,0.003937,-0.004788
Delivery_person_Ratings,-0.067881,1.000000,0.001495,-0.031262,0.033119,0.033839,-0.114943,-0.058092,0.050962,-0.331115,0.020250,-0.074999,0.019704,0.045145
Distance,0.023910,0.001495,1.000000,-0.059690,0.092853,-0.151594,0.000748,0.010466,0.002337,-0.002137,-0.017228,-0.027735,0.060672,-0.025890
Weather conditions,0.000213,-0.031262,-0.059690,1.000000,-0.039151,0.053056,-0.074197,-0.070516,0.028958,-0.196378,0.002030,0.012159,-0.022874,0.011012
Vehicle_condition,0.004788,0.033119,0.092853,-0.039151,1.000000,-0.091047,-0.095840,-0.079085,0.033030,-0.234398,-0.012041,-0.009175,0.027653,-0.012650
Type_of_vehicle,-0.004813,0.033839,-0.151594,0.053056,-0.091047,1.000000,-0.001445,-0.002717,0.003210,-0.000543,0.012671,0.025917,-0.052364,0.021725
multiple_deliveries,0.113326,-0.114943,0.000748,-0.074197,-0.095840,-0.001445,1.000000,0.141512,-0.059768,0.386497,0.015117,0.135501,-0.142997,0.002735
Festival,0.057092,-0.058092,0.010466,-0.070516,-0.079085,-0.002717,0.141512,1.000000,-0.019051,0.290046,-0.009587,0.120265,-0.082340,-0.031494
City,-0.057573,0.050962,0.002337,0.028958,0.033030,0.003210,-0.059768,-0.019051,1.000000,-0.166074,-0.006672,-0.049104,0.063316,-0.013023
Time_taken (min),0.292861,-0.331115,-0.002137,-0.196378,-0.234398,-0.000543,0.386497,0.290046,-0.166074,1.000000,0.033030,0.348888,-0.379932,0.024263


In [47]:
test

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Distance,Order_Date,Time_Order_picked,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Road_traffic_density_High,Road_traffic_density_Jam,Road_traffic_density_Low,Road_traffic_density_Medium
0,0x2318,COIMBRES13DEL01,30.0,4.632786,4.577196,30-03-2022,15:05,1,3.0,1,1.0,0,0,0,0,1,0
1,0x3474,BANGRES15DEL01,28.0,4.600000,12.554229,29-03-2022,20:35,1,0.0,1,1.0,0,0,0,1,0,0
2,0x9420,JAPRES09DEL03,23.0,4.500000,10.343089,10-03-2022,19:45,1,0.0,1,1.0,0,0,0,1,0,0
3,0x72ee,JAPRES07DEL03,21.0,4.800000,10.341179,02-04-2022,17:20,0,1.0,1,1.0,0,0,0,0,0,1
4,0xa759,CHENRES19DEL01,31.0,4.600000,12.456925,27-03-2022,18:40,1,2.0,1,1.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11394,0x6909,JAPRES01DEL01,35.0,4.600000,12.640017,27-03-2022,21:45,1,1.0,1,1.0,0,0,0,1,0,0
11395,0x443b,JAPRES11DEL01,33.0,4.900000,1.149391,11-03-2022,11:45,1,1.0,1,1.0,0,0,1,0,0,0
11396,0x1ea5,SURRES11DEL03,30.0,4.632786,6.982700,11-03-2022,21:05,1,3.0,1,1.0,0,0,0,0,1,0
11397,0x22d4,VADRES03DEL02,27.0,4.700000,15.100142,06-03-2022,18:40,1,0.0,1,0.0,0,0,0,0,0,1


In [48]:
test.corr()

,Delivery_person_Age,Delivery_person_Ratings,Distance,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Road_traffic_density_High,Road_traffic_density_Jam,Road_traffic_density_Low,Road_traffic_density_Medium
Delivery_person_Age,1.000000,-0.056157,0.000158,0.017715,0.001942,-0.020898,0.108888,0.039155,-0.067518,-0.002307,-0.010585,0.020154,-0.009512
Delivery_person_Ratings,-0.056157,1.000000,-0.038483,-0.052817,0.034132,0.010444,-0.118740,-0.046460,0.057001,0.021486,-0.083679,0.018996,0.054111
Distance,0.000158,-0.038483,1.000000,0.021987,0.078903,-0.085507,0.013781,-0.008228,0.006778,-0.012057,-0.027396,0.060749,-0.029998
Weather conditions,0.017715,-0.052817,0.021987,1.000000,0.037902,-0.030729,-0.068877,-0.077422,0.021488,0.016757,-0.021543,0.002988,0.008279
Vehicle_condition,0.001942,0.034132,0.078903,0.037902,1.000000,-0.105432,-0.119891,-0.105376,0.029232,0.008860,-0.020840,0.035677,-0.023540
Type_of_vehicle,-0.020898,0.010444,-0.085507,-0.030729,-0.105432,1.000000,0.006370,-0.008727,-0.008404,0.014769,0.029949,-0.060744,0.025360
multiple_deliveries,0.108888,-0.118740,0.013781,-0.068877,-0.119891,0.006370,1.000000,0.124013,-0.050926,0.003200,0.159237,-0.154899,-0.000804
Festival,0.039155,-0.046460,-0.008228,-0.077422,-0.105376,-0.008727,0.124013,1.000000,-0.009372,0.013944,0.105837,-0.083029,-0.030990
City,-0.067518,0.057001,0.006778,0.021488,0.029232,-0.008404,-0.050926,-0.009372,1.000000,0.004217,-0.050105,0.055448,-0.010863
Road_traffic_density_High,-0.002307,0.021486,-0.012057,0.016757,0.008860,0.014769,0.003200,0.013944,0.004217,1.000000,-0.218772,-0.243131,-0.185252


In [49]:
feat_cols =  [i for i in train.columns if i not in ['ID','Delivery_person_ID','Order_Date','Time_Order_picked','Road_traffic_density_High','Time_taken (min)']]
#train1 = train.drop(['ID','Delivery_person_ID','Order_Date','Time_Order_picked','Road_traffic_density_High','Time_taken (min)'], axis =1)
#test1 = test.drop(['ID','Delivery_person_ID','Order_Date','Time_Order_picked','Road_traffic_density_High'], axis =1)

In [50]:
train[feat_cols]

,Delivery_person_Age,Delivery_person_Ratings,Distance,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Road_traffic_density_Jam,Road_traffic_density_Low,Road_traffic_density_Medium
0,37.0,4.90000,2.297864,1,2.0,1,0.0,0,1,0,0,0
1,34.0,4.50000,14.836986,1,2.0,1,1.0,0,0,1,0,0
2,37.0,4.40000,4.565927,1,0.0,1,2.0,0,0,0,0,0
3,28.0,4.90000,2.277609,1,1.0,1,1.0,0,1,0,1,0
4,37.0,4.60000,7.843600,1,0.0,1,1.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
45588,34.0,4.90000,5.700149,0,2.0,1,0.0,0,0,0,0,1
45589,20.0,4.80000,6.951888,1,1.0,1,1.0,0,0,0,0,1
45590,30.0,4.63378,4.576893,0,3.0,1,1.0,0,0,0,1,0
45591,23.0,4.90000,16.086837,0,2.0,1,1.0,0,0,1,0,0


In [51]:
test[feat_cols]

,Delivery_person_Age,Delivery_person_Ratings,Distance,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Road_traffic_density_Jam,Road_traffic_density_Low,Road_traffic_density_Medium
0,30.0,4.632786,4.577196,1,3.0,1,1.0,0,0,0,1,0
1,28.0,4.600000,12.554229,1,0.0,1,1.0,0,0,1,0,0
2,23.0,4.500000,10.343089,1,0.0,1,1.0,0,0,1,0,0
3,21.0,4.800000,10.341179,0,1.0,1,1.0,0,0,0,0,1
4,31.0,4.600000,12.456925,1,2.0,1,1.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11394,35.0,4.600000,12.640017,1,1.0,1,1.0,0,0,1,0,0
11395,33.0,4.900000,1.149391,1,1.0,1,1.0,0,0,0,0,0
11396,30.0,4.632786,6.982700,1,3.0,1,1.0,0,0,0,1,0
11397,27.0,4.700000,15.100142,1,0.0,1,0.0,0,0,0,0,1


In [52]:
sc = MinMaxScaler()
sc.fit(train[feat_cols])

MinMaxScaler()

In [53]:
train_transformed = sc.transform(train[feat_cols])
test_transformed = sc.transform(test[feat_cols])

In [54]:
train_transformed = pd.DataFrame(train_transformed, columns = feat_cols)
test_transformed = pd.DataFrame(test, columns = feat_cols)

In [55]:
train_transformed

,Delivery_person_Age,Delivery_person_Ratings,Distance,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Road_traffic_density_Jam,Road_traffic_density_Low,Road_traffic_density_Medium
0,0.628571,0.780000,0.000060,1.0,0.666667,1.0,0.000000,0.0,1.0,0.0,0.0,0.0
1,0.542857,0.700000,0.000697,1.0,0.666667,1.0,0.333333,0.0,0.0,1.0,0.0,0.0
2,0.628571,0.680000,0.000175,1.0,0.000000,1.0,0.666667,0.0,0.0,0.0,0.0,0.0
3,0.371429,0.780000,0.000059,1.0,0.333333,1.0,0.333333,0.0,1.0,0.0,1.0,0.0
4,0.628571,0.720000,0.000342,1.0,0.000000,1.0,0.333333,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0.542857,0.780000,0.000233,0.0,0.666667,1.0,0.000000,0.0,0.0,0.0,0.0,1.0
45589,0.142857,0.760000,0.000296,1.0,0.333333,1.0,0.333333,0.0,0.0,0.0,0.0,1.0
45590,0.428571,0.726756,0.000176,0.0,1.000000,1.0,0.333333,0.0,0.0,0.0,1.0,0.0
45591,0.228571,0.780000,0.000761,0.0,0.666667,1.0,0.333333,0.0,0.0,1.0,0.0,0.0


In [56]:
test_transformed

,Delivery_person_Age,Delivery_person_Ratings,Distance,Weather conditions,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Road_traffic_density_Jam,Road_traffic_density_Low,Road_traffic_density_Medium
0,30.0,4.632786,4.577196,1,3.0,1,1.0,0,0,0,1,0
1,28.0,4.600000,12.554229,1,0.0,1,1.0,0,0,1,0,0
2,23.0,4.500000,10.343089,1,0.0,1,1.0,0,0,1,0,0
3,21.0,4.800000,10.341179,0,1.0,1,1.0,0,0,0,0,1
4,31.0,4.600000,12.456925,1,2.0,1,1.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11394,35.0,4.600000,12.640017,1,1.0,1,1.0,0,0,1,0,0
11395,33.0,4.900000,1.149391,1,1.0,1,1.0,0,0,0,0,0
11396,30.0,4.632786,6.982700,1,3.0,1,1.0,0,0,0,1,0
11397,27.0,4.700000,15.100142,1,0.0,1,0.0,0,0,0,0,1


In [57]:
x = train_transformed[feat_cols]
y = train['Time_taken (min)']
trainx, testx, trainy, testy = train_test_split(x, y, test_size=0.2, random_state=42)

In [58]:
model = LinearRegression()
model.fit(trainx, trainy)
#y_pred = model.predict(testx)
#print(metrics.accuracy_score(testy, output))
score = 100*metrics.r2_score(testy, model.predict(testx))
score

53.895318413385795

In [59]:
regressor = RandomForestRegressor(n_estimators = 500, random_state = 0, max_depth = 10)
regressor.fit(trainx, trainy)
#y_pred = regressor.predict(testx)
score = 100*metrics.r2_score(testy, regressor.predict(testx))
score

77.18987154691878

In [60]:
reg = ensemble.GradientBoostingRegressor(n_estimators = 500, max_depth = 5)
reg.fit(trainx, trainy)
score = 100*metrics.r2_score(testy, reg.predict(testx))
score

77.00465481706681

In [61]:
#regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(x, y)
rf = regressor.predict(test_transformed)
#r2_score(testy, y_pred)

In [62]:
reg.fit(x, y)
xgb = reg.predict(test_transformed)

In [63]:
di = {}
di['ID'] = test['ID']
di['rf'] = rf
di['xgb'] = xgb
di = pd.DataFrame(di)
di

,ID,rf,xgb
0,0x2318,46.198749,44.138539
1,0x3474,48.867325,50.947642
2,0x9420,48.867325,50.947642
3,0x72ee,44.982401,47.583414
4,0xa759,45.193727,46.090922
...,...,...,...
11394,0x6909,46.349778,44.255119
11395,0x443b,46.069726,46.358230
11396,0x1ea5,46.198749,44.138539
11397,0x22d4,35.639232,39.351712


In [64]:
di['Time_taken (min)'] = (di['rf']+di['xgb'])/2.0
di['Time_taken (min)'] = di['Time_taken (min)'].round()
di.drop(['rf', 'xgb'], axis = 1, inplace = True)
di

,ID,Time_taken (min)
0,0x2318,45.0
1,0x3474,50.0
2,0x9420,50.0
3,0x72ee,46.0
4,0xa759,46.0
...,...,...
11394,0x6909,45.0
11395,0x443b,46.0
11396,0x1ea5,45.0
11397,0x22d4,37.0


In [65]:
#Output
Output=pd.DataFrame(di) 
Output.to_csv('submission3.csv',index=False)
from IPython.display import FileLink
FileLink('submission3.csv')

/kaggle/working/submission3.csv